# Load single graph data
此為單一結構物的 graph data，以類似 dictionary 的結構存取

- x: node features
- edge_index: node pairs
- edge_attr: edge features
- Acceleration, Velocity, Desplacement, Moment_Z, Shear_Y, 原始反應歷時資料的 size 皆為 [2000, 8]，因反應立時的取樣頻率為 0.05，最大長度設定為 100 秒，故為 2000 步，8 則是設定最高能預測的建築物樓高。
- ground_motion (mm/$s^2$): [time_steps]
- time_steps: 未經前處理的 ground motion 步數
- sample_rate: 地表加速度資料的取要頻率 (為反應歷時取樣頻率的 10 倍)
- ground_motion_name: 地震紀錄名

In [1]:
import torch
from torch import nn
sample_path = "./Data/Linear_Analysis/eval/ChiChi/structure_401/structure_graph.pt"
sample_graph = torch.load(sample_path)
print(sample_graph)

Data(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], Acceleration=[2000, 8], Velocity=[2000, 8], Displacement=[2000, 8], Moment_Z=[147, 2000, 6], Shear_Y=[147, 2000, 6], ground_motion=[6000], time_steps=6000, sample_rate=0.005, ground_motion_name='1999.09.21_01.47.159_Chichi_TCU011_A900_90.0')


# Load Data with GraphDataset
- 輸入欲載入的資料集路徑與反應種類
- GraphDataset 會進行初步前處理，將結構反應歷時 y 轉為 3 維向量，方便 LSTM model 使用，並存下該結構物樓高與反應種類

In [2]:
from Utils.dataset import GraphDataset

ChiCHi_folder_dir = "./Data/Linear_Analysis/eval/ChiChi"
NGAWest2_folder_dir = "./Data/Linear_Analysis/eval/NGAWest2"

sampleDataset = GraphDataset(folder_path_list = [ChiCHi_folder_dir, NGAWest2_folder_dir], response_type = "Velocity", numOfData_per_folder = 5)

print(sampleDataset[0])
print(f"\n # of data:{len(sampleDataset)}")

Loading data from ./Data/Linear_Analysis/ChiChi


100%|██████████| 5/5 [00:01<00:00,  3.80it/s]


Loading data from ./Data/Linear_Analysis/NGAWest2


100%|██████████| 5/5 [00:01<00:00,  3.85it/s]

  number of effective data: 10 
Data(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], ground_motion=[1, 20000], time_steps=6000, sample_rate=0.005, ground_motion_name='1999.09.21_01.47.159_Chichi_TCU011_A900_90.0', response_type='Velocity', story=6, y=[1, 2000, 8])

 # of data:10


## Normalization dictionary
資料輸入模型之前需要先經過 normalization 將每個 feature dimention 調整到 [-1, 1]，因此會需要用到各個 feature dimention 的極大值

In [3]:
print(f"(source) normalization state: {sampleDataset.source_norm_state}")
print(f"\n(target) normalization state: {sampleDataset.target_norm_state}")
normalized_item_dict = sampleDataset.get_normalized_item_dict()
print("\nnormalization dictionary: \n", normalized_item_dict)


(source) normalization state: False

(target) normalization state: False

normalization dictionary: 
 {'x': {'XYZ_gridline_num': tensor(8.), 'XYZ_grid_index': tensor(7.), 'period': tensor(1.2711), 'DOF': tensor(1.), 'mass': tensor(0.0252), 'XYZ_inertia': tensor(255288.), 'XYZ_mode_shape': tensor(2.1000)}, 'ground_motion': tensor(6140.5513), 'y': tensor(2678.), 'edge_attr': {'S_y': tensor(3687090.), 'S_z': tensor(3687090.), 'area': tensor(30774.), 'element_length': tensor(8000.)}, 'response_type': 'Velocity'}


### Noramalize
以各個 feature dimention 的極大值來 normalize 資料

In [4]:
sampleDataset.normalize_source(normalized_item_dict)
sampleDataset.normalize_target(normalized_item_dict)
print(f"(source) normalization state: {sampleDataset.source_norm_state}")
print(f"\n(target) normalization state: {sampleDataset.target_norm_state}")

(source) normalization state: True

(target) normalization state: True


### Denormalize
模型計算完後再將計算結果 demormalize 回原本的 scale

In [5]:
sampleDataset.denormalize_source(normalized_item_dict)
sampleDataset.denormalize_target(normalized_item_dict)
print(f"(source) normalization state: {sampleDataset.source_norm_state}")
print(f"\n(target) normalization state: {sampleDataset.target_norm_state}")

(source) normalization state: False

(target) normalization state: False


# DataLoader (PyG)
以 DataLoader 作為迭代器，訓練時 batch size 可以任意調整，shuffle=True；預測時 batch size = 1 ，shuffle=False

In [6]:
from torch_geometric.loader import DataLoader as GraphDataLoader
exclude_keys=["sample_rate", "ground_motion_name", "response_type"]
sample_loader = GraphDataLoader(sampleDataset, batch_size=1, shuffle=False, )

for (i, data) in enumerate(sample_loader):
    print('=======')
    print(f'mini batch {i}:' )
    # data = data.to("cuda")
    print(data)
    print()

mini batch 0:
DataBatch(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[147], ptr=[2])

mini batch 1:
DataBatch(x=[210, 15], edge_index=[2, 904], edge_attr=[904, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[210], ptr=[2])

mini batch 2:
DataBatch(x=[168, 15], edge_index=[2, 742], edge_attr=[742, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[168], ptr=[2])

mini batch 3:
DataBatch(x=[105, 15], edge_index=[2, 444], edge_attr=[444, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[105], ptr=[2])

mini batch 4:
DataBatch(x=[245, 15], edge_index=[2, 1116], edge_attr=[1116, 6], 

## Pack and Pad

In [7]:
compression_rate = 20
data = next(iter(sample_loader))
ground_motion = data.ground_motion.reshape(-1, int(data.ground_motion.size(1)/compression_rate), compression_rate)
ground_motion = ground_motion.to("cuda")

packed_length_list = data.time_steps / compression_rate
total_length = ground_motion.size(1)
print('=======')
print("mini-batch:")
print(data)
print()

print('=======')
print("size of compressed ground motion:")
print(ground_motion.size())
print()
print("PPS length list:")
print(packed_length_list)
print()

packed_x = nn.utils.rnn.pack_padded_sequence(ground_motion, lengths = packed_length_list, batch_first=True, enforce_sorted=False)
seq_unpacked_x, lens_unpacked = nn.utils.rnn.pad_packed_sequence(packed_x, batch_first=True, total_length = total_length)

print('=======')
print("size of unpacked ground motion:")
print(seq_unpacked_x.size())
print()
print("check x has unpacked to original format")
print(torch.equal(seq_unpacked_x, ground_motion))
print()
print("unpack length list:")
print(lens_unpacked)


mini-batch:
DataBatch(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[147], ptr=[2])

size of compressed ground motion:
torch.Size([1, 1000, 20])

PPS length list:
tensor([300.])

size of unpacked ground motion:
torch.Size([1, 1000, 20])

check x has unpacked to original format
True

unpack length list:
tensor([300])
